In [2]:
import pandas as pd
import hygia as hg
import pickle

/home/anapaula/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-28 20:08:59.580652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 20:08:59.659490: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-28 20:08:59.659504: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on you

# Classes instanciations

NOTE: Please check if the model_path matches your path

In [3]:
pre_process_data = hg.PreProcessData()
augment_data = hg.AugmentData(country="MEXICO")
feature_engineering = hg.FeatureEngineering()
annotate_data = hg.AnnotateData()
new_rf_model = hg.RandomForestModel()

# Load Data

NOTE: Please check if the file_path matches your data

In [4]:
file_path = '../data/tmp/AI_LATA_ADDRESS_MEX_modificado.csv'
df = pd.read_csv(file_path, sep='¨', nrows=None, engine='python')

# Add new columns

1. Concatenate address
2. All features columns:
    - Key Smash
    - Regex
    - Word Embedding

NOTE: Please check if the columns names matches your data

In [5]:
concatened_column_name = 'concat_STREET_ADDRESS_1_STREET_ADDRESS_2'
df = pre_process_data.pre_process_data(df, ['STREET_ADDRESS_1', 'STREET_ADDRESS_2'], concatened_column_name)
df = feature_engineering.extract_features(df, concatened_column_name)

aliases indified: concat_STREET_ADDRESS_1_STREET_ADDRESS_2 -> ['STREET_ADDRESS_1', 'STREET_ADDRESS_2']
handle null values in the column concat_STREET_ADDRESS_1_STREET_ADDRESS_2


# Check new columns names

In [6]:
all_features_columns = [col for col in df if col.startswith('feature_ks') or col.startswith('feature_we') or col.startswith('feature_re')]
all_features_columns

['feature_ks_count_sequence_squared_vowels_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_count_sequence_squared_consonants_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_count_sequence_squared_special_characters_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_ratio_of_numeric_digits_squared_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_average_of_char_count_squared_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_0_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_1_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_2_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_3_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_4_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_5_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_6_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_7_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_8_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_9_concat_STREET_A

# Annotate data

In [7]:
key_smash_thresholds = {
    'count_sequence_squared_vowels': 1.00,
    'count_sequence_squared_consonants': 1.999,
    'count_sequence_squared_special_characters': 2.2499,
    'ratio_of_numeric_digits_squared': 2.9,
    'average_of_char_count_squared': 2.78,
}

df = annotate_data.annotate_data(df, concatened_column_name, key_smash_thresholds)
df

valid                             1346998
key_smash                             739
contains_email                        569
contains_exactly_the_word_test        177
only_special_characters               144
contains_exactly_the_word_dell        125
only_numbers                          106
only_one_char                          14
contains_exactly_invalid_words         10
is_substring_of_column_name             3
contains_date                           1
empty                                   1
Name: target, dtype: int64

In [8]:
df['target'].value_counts()

valid                             2514338
key_smash                            1762
only_special_characters              1291
contains_email                       1048
contains_exactly_the_word_test        667
contains_exactly_the_word_dell        553
only_one_char                         287
only_numbers                          239
empty                                  71
contains_exactly_invalid_words         26
is_substring_of_column_name            12
contains_date                           2
Name: target, dtype: int64

# Experiment: retrain model

In [9]:
scores = new_rf_model.train_and_get_scores(df, concatened_column_name, all_features_columns)
scores

{'accuracy': 0.9885931558935361,
 'precision': 0.9822222222222222,
 'recall': 0.9778761061946902,
 'f1': 0.9800443458980044}

# Predict using pre-trained model

In [10]:
df['prediction'] = new_rf_model.predict(df[all_features_columns].values)
df.drop_duplicates(subset=[concatened_column_name])['prediction'].value_counts()

/home/anapaula/.local/lib/python3.8/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.0    1344545
1.0       4342
Name: prediction, dtype: int64

# Save model and predicted data

In [11]:
new_rf_model.export_model('../data/models/RandomForest_Ksmash_WordEmbedding_Regex_Enrichments.pkl')

In [14]:
df[df['prediction'] == 1][[concatened_column_name, 'prediction']].drop_duplicates(subset=[concatened_column_name]).to_csv('../data/tmp/prediction_enrichments.csv')

In [13]:
df['prediction'] = df['target'] == 'key_smash'
df['prediction_model'] = df['prediction']
df[df['prediction_model']!=df['prediction']][[concatened_column_name, 'target', 'prediction_model', 'prediction']] \
    .drop_duplicates(subset=[concatened_column_name]) \
    .to_csv('../data/tmp/prediction_better_with_model.csv')